In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv(os.path.join("..", "..", "data", "retail", "events.csv"))

In [3]:
data.reset_index(drop=True, inplace=True)
data["event"].replace("view", 0, inplace=True)
data["event"].replace("addtocart", 1, inplace=True)
data["event"].replace("transaction", 2, inplace=True)

In [6]:
data.event.value_counts()

0    2664312
1      69332
2      22457
Name: event, dtype: int64

In [ ]:
data.columns = data.columns.str.replace("ts", "timestamp")
data.columns = data.columns.str.replace("visitorid", "session")
data.columns = data.columns.str.replace("event", "type")
data.columns = data.columns.str.replace("itemid", "aid")
data = data[["session", "aid", "timestamp", "type"]]

In [ ]:
max_session_type = data.groupby(["session", "aid"])["type"].max().rename("max_type")

In [ ]:
data = data.join(max_session_type, ["session", "aid"])

In [ ]:
data = data[data["type"] == data["max_type"]]

In [ ]:
data.reset_index(drop=True, inplace=True)
data.drop(columns="max_type", inplace=True)

In [ ]:
click_weight, cart_weight, purchase_weight = data.shape[0] / data["type"].value_counts()

In [ ]:
data = data.join(
    data.groupby(["session", "aid"])["type"].count().rename("count"),
    on=["session", "aid"],
)

In [ ]:
data.drop_duplicates(["session", "aid"], inplace=True)

In [ ]:
click_border = click_weight * (cart_weight // click_weight)
cart_border = cart_weight * (purchase_weight // cart_weight)

In [ ]:
data["weight"] = np.where(
    data["type"] == 0,
    np.minimum(click_border, data["count"] * click_weight),
    np.where(
        data["type"] == 1,
        np.minimum(cart_border, data["count"] * cart_weight),
        data["count"] * purchase_weight,
    ),
)

In [ ]:
data.drop(columns=["type", "count"], inplace=True)

In [ ]:
data.to_parquet(
    os.path.join("..", "..", "data", "retail", "interactions.parquet"), index=False
)